# Feature Matrix Generation

This chapter introduces similarity metrics for string comparison. The metrics to be used for calculating its similarity is decided for each attribute of the DataFrame built in the previous chapters. As a result of this chapter, the feature matrix will be derived.

## Table of Contents

- [Data Takeover](#Data-Takeover)
- [Object Distance and Similarity](#Object-Distance-and-Similarity)
- [Library TextDistance](#Library-TextDistance)
- [Similarity Metrics on Attribute Level](#Similarity-Metrics-on-Attribute-Level)
    - [coordinate](#coordinate)
    - [corporate](#corporate)
    - [doi](#doi)
    - [edition](#edition)
    - [exactDate](#exactDate)
    - [format](#format)
    - [isbn](#isbn)
    - [musicid](#musicid)
    - [part](#part)
    - [person](#person)
    - [pubinit](#pubinit)
    - [scale](#scale)
    - [ttlfull](#ttlfull)
    - [volumes](#volumes)
- [Feature Base](#Feature-Base)
- [Summary](#Summary)
- 

## Data Takeover

Swissbib's raw data of the goldstandard has been processed in chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb). As the first step of this chapter, this data is read in for further processing to the feature matrix and target vector for the subsequent machine learning model chapters.

In [1]:
import os
import pandas as pd
import pickle as pk

path_goldstandard = './daten_goldstandard'

# Restore metadata so far
with open(os.path.join(path_goldstandard, 'columns_metadata.pkl'), 'rb') as handle:
    columns_metadata_dict = pk.load(handle)

# Restore results so far
df_feature_base = pd.read_pickle(os.path.join(path_goldstandard, 'feature_base_df.pkl'),
                                 compression=None)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_feature_base.columns)

df_feature_base.head()

,duplicates,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,corporate_full_x,corporate_full_y,doi_x,doi_y,edition_x,edition_y,exactDate_x,exactDate_y,format_prefix_x,format_prefix_y,format_postfix_x,format_postfix_y,isbn_x,isbn_y,musicid_x,musicid_y,part_x,part_y,person_100_x,person_100_y,person_700_x,person_700_y,person_245c_x,person_245c_y,pubinit_x,pubinit_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,volumes_x,volumes_y
0,1,,,,,,,,,,,2009uuuu,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane1775-1817(de-588)118505173,"grawechristian, graweursula","grawechristian, graweursula",jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem englischen übersetzt von...,reclam jun.,reclam jun.,,,"emma, roman","emma, roman",,,600 s.,600 s.
1,1,,,,,,,,,,,2009uuuu,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane1775-1817(de-588)118505173,"grawechristian, graweursula",,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem engl. übers. von ursula ...,reclam jun.,reclam,,,"emma, roman",emma,,,600 s.,600 s.
2,1,,,,,,,,,,,2009uuuu,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane,"grawechristian, graweursula",,jane austen ; aus dem englischen übersetzt von...,jane austen,reclam jun.,reclam,,,"emma, roman","emma, roman",,,600 s.,600 s.
3,1,,,,,,,,,,,2009uuuu,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane1775-1817(de-588)118505173,,"grawechristian, graweursula",jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem englischen übersetzt von...,reclam,reclam jun.,,,emma,"emma, roman",,,600 s.,600 s.
4,1,,,,,,,,,,,2009uuuu,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane1775-1817(de-588)118505173,,,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem engl. übers. von ursula ...,reclam,reclam,,,emma,emma,,,600 s.,600 s.


In [2]:
print('Number of rows labelled as duplicates', len(df_feature_base[
    df_feature_base.duplicates==1]))
print('Number of rows labelled as uniques', len(df_feature_base[
    df_feature_base.duplicates==0]))
print('Total number of rows in DataFrame', df_feature_base.shape[0],
      'number of columns', df_feature_base.shape[1])

Number of rows labelled as duplicates 1473
Number of rows labelled as uniques 259260
Total number of rows in DataFrame 260733 number of columns 39


In [3]:
print('Part of duplicates (1) on uniques (2) in units of [%]')
print(100*df_feature_base.duplicates.value_counts(normalize=True))

Part of duplicates (1) on uniques (2) in units of [%]
0    99.435054
1     0.564946
Name: duplicates, dtype: float64


The ratio of duplicate records in the full training data has a percentage value of below 0.6%. This is very low and will affect the training of the model. During the training process, the model will find more pairs of unique records ($\texttt{duplicates}=0$) than pairs of duplicates ($\texttt{duplicates}=1$). Undersampling of the amount of unique pairs might be necessary as a consequence and will be decided during model training.

## Object Distance and Similarity

A mathematical idea of distance and similarity is needed for understanding object pair comparison. This section starts with a motivation for calculating similarities and afterwards gives a very basic definition of the two central terms. The text of this section is a summary of [[Chri2012](./A_References.ipynb#chri2012)].

The attributes to be used for pair comparison may contain values of poor quality. The quality originates in the way the data has been entered at the very source. Manual data entry may suffer from mistyping, automatically scanned data may suffer from insufficiencies of the scanned base material or the recognition algorithm in the optical character recognition (OCR) processing. The basic step of a deduplication process is to identify the probability of two strings of a pair to be a pair of duplicates. This is done by calculating a similarity value between the two strings compared, rather than using an exact comparison function. Based on this common similarity value for an attribute pair, their being duplicates can be decided.

The term similarity is strongly coupled to the term of distance of two values of an attribute. Mathematically, a distance can be explained with the help of a distance function. A _distance function_ or _distance metric_ $dist(o_i, o_j)$ between two points or data objects $o_i$ and $o_j$ must fulfill four requirements.

1. $dist(o_i, o_i)=0$, the distance from an object to itself is zero.
- $dist(o_i, o_j)\ge 0$, the distance between two objects is a non-negative number.
- $dist(o_i, o_j)=dist(o_j, o_i)$, the distance between two objects is symmetric.
- $dist(o_i, o_j)\le dist(o_i, o_k)+dist(o_k, o_j)$, the triangular inequality must hold. It states that the direct distance beween two objects is never larger than the combined distance when going through a third object.

A distance value expresses the dissimilarity $d$ of two objects [[HanK2012](./A_References.ipynb#hank2012)] and can therefore be converted into a similarity value $s$, calculating $s = \frac{1}{d}$, assuming $d\gt 0$. Alternatively, assuming the distance value is normalised $0\le d\le 1$, the similarity value can be calculated to $s = 1-d$. A _similarity function_ $sim(a_i, aj)$ between two attributes which can be strings, numbers, dates, geographic locations, text, XML documents, etc. fulfills the general requirements.

1. $sim(a_i, a_i)=1$, the result of comparing a value with itself is an exact similarity.
- $sim(a_i, a_j)=0$, the similarity of values that are completely different from each other is 0. What accounts for 'complete different' depends upon the type of data that are compared.
- $0\lt sim(a_i, a_j)\lt 1$, an approximate similarity between exact similarity and total dissimilarity is calculated if two attribute values are somewhat similar to each other. What accounts for 'somewhat different' depends upon the type of data that are compared.

The dissimilarity between two objects $o_i$ and $o_j$ can be computed based on the ratio of mismatches,
$$
d(o_i, o_j) = \frac{p-m}{p},
$$
where $m$ is the number of matching attributes and $p$ is the total number of attributes describing the objects [[HanK2012](./A_References.ipynb#hank2012)]. Thus the similarity between two objects can be computed as
$$
sim(o_i, o_j) = 1 - d(o_i, o_j) = \frac{m}{p}.
$$

For data deduplication, a comparison function needs to be tailored to the type of underlying data. Although there is a correspondence between a similarity function and the mathematical concept of a distance function, not all known and implemented similarity comparison functions used for string pair comparison fulfill the requirements of a distance function. Some similarity functions are not symmetric, others do not fulfill the triangular inequality. Decision taking on the best similarity function for a string pair, will be based on the effect, a similarity function has for the purpose needed. In the case of this capstone project, this purpose is its capability to contribute to the prediction whether a pair of records is a duplicate or different.

## Library TextDistance

An internet research on string distance calculation with Python has revealed libraries [[StSi](./A_References.ipynb#stsi)], [[TeDi](./A_References.ipynb#tedi)] and seperate code snippets for distinct algorithms. After trying the referenced libraries and a downloaded code snippet for a Smith Waterman similarity [[SmWa](./A_References.ipynb#smwa)], the text distance library [[TeDi](./A_References.ipynb#tedi)] has been decided to be the best decision for this capstone project. The decision is based on the GitHub statistics of stars and the date of the latest pull requests, indicating its popularity and maintenance activity of the library. A look at the API of the library, reveals the Python library to be a complete implementation (compared to suggestions of similarity metrics in [[Chri2012](./A_References.ipynb#chri2012)]) and easy to use.

In [4]:
# Install textdistance Python library - if not done, yet.
! pip install textdistance

For using the library, see documentation in [[TeDi](./A_References.ipynb#tedi)]. For the purposes of this chapter, function $\texttt{.normalized}\_\texttt{similarity()}$ of an instantiated textdistance object will be used.

In [5]:
import textdistance as tedi

With the code line above, the library is imported for application in this chapter. In appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb) the effect of the similarity metrics of the library are compared for better understanding of their specific behaviour. This comparison for each attribute is the basis of deciding the best similarity metric available for an attribute pair.

## Similarity Metrics on Attribute Level

In this section, the decision for calculating the similarity metric for each attribute of the raw data is documented based on appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb) and implemented. The implementation is applied on a pair of attributes of different records, resulting in a new attribute of the final feature matrix. A general function $\texttt{build_delta_feature}$ is provided by the code file [data_preparation_funcs.py](./data_preparation_funcs.py) for transforming two attributes into their feature attribute holding their similarity value.

In [6]:
import data_preparation_funcs as dpf

In [7]:
columns_metadata_dict['similarity_metrics'] = {}
columns_metadata_dict['columns_for_comparison'] = []

### coordinate

As discussed in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{coordinate}$ holds coordinates of maps. To decide whether a map covers the same geographical range, a metric will be chosen that compares the coordinate number digits from left to right. The more digits are found to be the equal, the higher the similarity value is calculated. The comparison stops with the first digit pair that is unequal. This algorithm is satisfyed by the LCS (Longest Common Substring comparison) algorithm and generates the wanted resault, see appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb).

In [8]:
attribute = 'coordinate'

columns_metadata_dict['similarity_metrics'][attribute+'_E'] = tedi.LCSStr()
columns_metadata_dict['similarity_metrics'][attribute+'_N'] = tedi.LCSStr()

ne_values = ['_E', '_N']

for ne in ne_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+ne,
        columns_metadata_dict['similarity_metrics'][attribute+ne],
        columns_metadata_dict)

Looking at some samples of the feature matrix reveals a good match to the expectations.

In [9]:
uniques, uniques_len = {}, {}

for ne in ne_values :
    uniques[attribute+ne], uniques_len[attribute+ne] = dpf.determine_similarity_values(
        df_feature_base, attribute+ne)

coordinate_E values range [0.    0.375 0.5   0.625 0.875 1.   ]
coordinate_N values range [0.    0.375 0.5   0.75  0.875 1.   ]


In [10]:
position = 3

for ne in ne_values :
    dpf.show_samples_interval(
        df_feature_base, attribute+ne,
        uniques[attribute+ne][uniques_len[attribute+ne]-position],
        uniques[attribute+ne][uniques_len[attribute+ne]-position+1]
    )

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y
54109,0,0.875,e0080855,e0080851
865,1,0.875,e0080851,e0080855
54046,0,0.625,e0080855,e0080900
68413,0,0.875,e0080855,e0080851
869,1,0.875,e0080855,e0080851


0.625 <= coordinate_E_delta <= 0.875


,duplicates,coordinate_N_delta,coordinate_N_x,coordinate_N_y
68353,0,0.75,n0460826,n0460833
94855,0,0.75,n0460821,n0460833
68401,0,0.75,n0460826,n0460833
53737,0,0.75,n0460826,n0460833
68360,0,0.75,n0460826,n0460833


0.75 <= coordinate_N_delta <= 0.875


The samples above show the wanted similarity behaviour for value ranges greater than 0. The metric has the weakness, though, that empty coordinate values, e.g. for bibliographical units other than maps, have each been calculated to a similarity of 0. Some samples for duplicates in the training data are shown below.

In [11]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1],
    attribute+'_E', uniques[attribute+'_E'][0], uniques[attribute+'_E'][1], 10)

,duplicates,coordinate_E_delta,coordinate_E_x,coordinate_E_y
1153,1,0.0,,
84,1,0.0,,
30,1,0.0,,
295,1,0.0,,
1264,1,0.0,,
1255,1,0.0,,
1349,1,0.0,,
977,1,0.0,,
767,1,0.0,,
394,1,0.0,,


0.0 <= coordinate_E_delta <= 0.375


This downside shall be avoided by marking pairs of missing coordinate values on both sides with a special value of -1, which will point out to the models to be fitted the special case of missing information in a row. The implementation of this logic is done in function $\texttt{.mark}\_\texttt{both}\_\texttt{missing()}$.

In [12]:
for ne in ne_values :
    df_feature_base = dpf.mark_both_missing(df_feature_base, attribute+ne)

### corporate

Attribute $\texttt{corporate}$ is a collection of corporate names. The Monge-Elkan metric compares string tokens pairwise [[Chri2012](./A_References.ipynb#chri2012)] while the LCS metric searches for the longest common substring. Assessing the differences of these two metrics with the help of their values distribution in chapter [Features Discussion and Dummy Classifier Baseline](./4_FeatureDiscussionDummyBaseline.ipynb), reveals a better distribution behaviour for LCS. Therefore, the LCS metric will be chosen for this attribute.

In [13]:
attribute = 'corporate_full'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.StrCmp95()
# tedi.LCSStr()
# tedi.MongeElkan()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

Its $110$ part is sparsely filled and even its $710$ part comes along with a little more than $10\%$ of filling, only. The LCS metric generates a similarity of 1 for the cases where both strings of a pair are empty. Missing values on both sides may be an indicator for a pair of duplicates but due to the sparsely available information, it is a weak indicator. Therefore, the pairs with missing data on both sides of the pair, will be marked with the special value of -1.

In [14]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

corporate_full values range [0.         0.30357143 0.30414909 ... 0.97974684 0.99473684 1.        ]


In [15]:
df_feature_base = dpf.mark_both_missing(df_feature_base, attribute)

Some sample cases are shown below for both $\texttt{corporate}$ features.

In [16]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1],
    attribute, 0.0, 1.0, 20
)

,duplicates,corporate_full_delta,corporate_full_x,corporate_full_y
1310,1,0.787047,"schweiz, eidgenössisches topographisches bureau",eidgenössisches topographisches bureau
1416,1,1.000000,schweizerische normen-vereinigung,schweizerische normen-vereinigung
188,1,0.898000,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforsch...
1117,1,1.000000,interkantonale lehrmittelzentrale (luzern),interkantonale lehrmittelzentrale (luzern)
725,1,1.000000,"rundfunkchor, sächsische staatskapelle dresden","rundfunkchor, sächsische staatskapelle dresden"
183,1,1.000000,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforschung
1421,1,1.000000,schweizerische normen-vereinigung,schweizerische normen-vereinigung
201,1,1.000000,schweizerische gesellschaft für bildungsforschung,schweizerische gesellschaft für bildungsforschung
1257,1,0.000000,,springerlink (online service)
1152,1,1.000000,"interkantonale lehrmittelzentrale (rapperswil,...","interkantonale lehrmittelzentrale (rapperswil,..."


0.0 <= corporate_full_delta <= 1.0


In [17]:
position = uniques_len[attribute]//2 # Let's have a look in the middle range of the similarities.

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 20)

,duplicates,corporate_full_delta,corporate_full_x,corporate_full_y
149049,0,0.574090,"staatsoper (wien), wiener philharmoniker, chor","malpigli, annibale (bologna)"
148669,0,0.574084,arts florissants,"rundfunkchor (leipzig), staatskapelle (dresden)"
234545,0,0.574207,philharmonia orchestra (london),münchner bläserakademie


0.5740839243498818 <= corporate_full_delta <= 0.5742069057637079


### doi

Swissbib uses an explicit $\texttt{doi}$ attribute for its deduplication implementation. In chapter [Goldstandard and Data Preparation](./2_GoldstandardDataPreparation.ipynb), the real doi identifier has been isolated with the help of a preprocessing function $\texttt{.reduce}\_\texttt{to}\_\texttt{doi}\_\texttt{element()}$, see [Data Analysis](./1_DataAnalysis.ipynb). Attribute $\texttt{doi}$ contains a single string value. The Identity metric will be used for comparing the string values of a pair in a row, calculating a similarity value of 1.0 or 0.0 for each pair. If one list is empty a value of 0 is returned.

In [18]:
attribute = 'doi'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

df_feature_base['doi_delta'].unique()

array([1., 0.])

Some sample cases are shown below for each category of $\texttt{doi}\_\texttt{delta}$.

In [19]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

for doi_delta_value in df_feature_base['doi_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['doi_delta']==doi_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'doi', doi_delta_value, number_of_max_samples)
    print(f'doi_delta = {doi_delta_value}')

doi values range [0. 1.]


,duplicates,doi_delta,doi_x,doi_y
106641,0,1.0,,
63293,0,1.0,,
3267,0,1.0,,
121544,0,1.0,,
116532,0,1.0,,
2288,0,1.0,,
4139,0,1.0,,
122126,0,1.0,,
144410,0,1.0,,
164661,0,1.0,,


doi_delta = 1.0


,duplicates,doi_delta,doi_x,doi_y
65319,0,0.0,,10.5169/seals-376473
67716,0,0.0,,10.5169/seals-376961
32544,0,0.0,,10.5169/seals-376645
169639,0,0.0,,10.5169/seals-377218
145207,0,0.0,,10.5169/seals-377332
124148,0,0.0,,10.3931/e-rara-61897
34326,0,0.0,,10.5169/seals-376437
200044,0,0.0,10.1007/978-3-642-41698-9,
191720,0,0.0,,10.5169/seals-515343
10488,0,0.0,,10.5169/seals-376510


doi_delta = 0.0


In [20]:
# Let's have a look at some non-empty doi elements
df_doi_with_element = df_feature_base[df_feature_base.doi_x.apply(lambda x : len(x) > 0)]

for doi_delta_value in df_feature_base['doi_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['doi_delta']==doi_delta_value])
    )

    dpf.show_samples_distinct(df_doi_with_element, 'doi', doi_delta_value, number_of_max_samples)
    print(f'doi_delta = {doi_delta_value}')

,duplicates,doi_delta,doi_x,doi_y
1472,1,1.0,10.1055/b-005-143650,10.1055/b-005-143650
1251,1,1.0,10.1055/b-002-26639,10.1055/b-002-26639
1267,1,1.0,10.1093/cid/ciu795,10.1093/cid/ciu795
1193,1,1.0,10.1093/cid/cir669,10.1093/cid/cir669
1192,1,1.0,10.1093/cid/cir669,10.1093/cid/cir669
207553,0,1.0,10.1093/cid/cir669,10.1093/cid/cir669
1264,1,1.0,10.1093/ndt/gft319,10.1093/ndt/gft319
1470,1,1.0,10.1055/b-005-143650,10.1055/b-005-143650
166787,0,1.0,10.1007/978-3-642-41698-9,10.1007/978-3-642-41698-9
1190,1,1.0,10.1093/cid/cir669,10.1093/cid/cir669


doi_delta = 1.0


,duplicates,doi_delta,doi_x,doi_y
193829,0,0.0,10.5451/unibas-006503313,
208005,0,0.0,10.1093/ndt/gft319,
193867,0,0.0,10.5451/unibas-006503313,
255303,0,0.0,10.1055/b-005-143650,
167123,0,0.0,10.1093/cid/ciu795,
200079,0,0.0,10.1007/978-3-642-41698-9,
154222,0,0.0,10.5167/uzh-53042,10.3931/e-rara-50904
193950,0,0.0,10.5451/unibas-006503313,
190053,0,0.0,10.1055/b-002-26639,
207587,0,0.0,10.1093/cid/cir669,


doi_delta = 0.0


As can be seen above, a value of 1.0 is returned if both strings of a pair are empty. As the attribute filling of $\texttt{doi}$ is sparse, see chapter [Data Analysis](./1_DataAnalysis.ipynb), the $\texttt{doi}\_\texttt{delta}$ indicates strongly a duplicate for most rows. To avoid such misleading identity indication, function $\texttt{.mark}\_\texttt{both}\_\texttt{missing()}$ will be applyed to the attribute.

In [21]:
df_feature_base = dpf.mark_both_missing(df_feature_base, attribute)

### edition

The edition statement is a string value which may have several words. A Jaccard similarity is tried for this attribute.

In [22]:
attribute = 'edition'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [23]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

import numpy as np

edition_delta_uniques = np.sort(df_feature_base['edition_delta'].unique())
edition_delta_uniques_len = len(edition_delta_uniques)
print('edition values range', edition_delta_uniques[:30])

edition values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.4        0.5        0.6        1.        ]
edition values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.4        0.5        0.6        1.        ]


The comparison results in a wide number of distinct similarity values for the goldstandard data set. Below, some examples are shown.

In [24]:
position = edition_delta_uniques_len

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position],
    edition_delta_uniques[edition_delta_uniques_len-position+2], 10)

position = edition_delta_uniques_len//2

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)

,duplicates,edition_delta,edition_x,edition_y
220803,0,1.0,,
126322,0,1.0,,
88908,0,1.0,,
227207,0,1.0,,
92201,0,1.0,,
92356,0,1.0,,
144162,0,1.0,,
219144,0,1.0,,
205428,0,1.0,,
739,0,1.0,,


0.6 <= edition_delta <= 1.0


,duplicates,edition_delta,edition_x,edition_y
96476,0,0.0,1791,
90189,0,0.0,,3
163761,0,0.0,,1899
225563,0,0.0,,4
72728,0,0.0,,3
27763,0,0.0,5,
202380,0,0.0,,2
49334,0,0.0,,1
26566,0,0.0,,11
70423,0,0.0,8,1907


0.0 <= edition_delta <= 0.1428571428571428


,duplicates,edition_delta,edition_x,edition_y
53734,0,0.25,1,1905
53959,0,0.25,1,1994
195586,0,0.25,2,1926
54658,0,0.25,1,1943
102350,0,0.20,10,1928
191750,0,0.25,2,1928
102112,0,0.20,10,1994
174122,0,0.25,6,1869
248145,0,0.20,10425,4
257566,0,0.25,5,1905


0.19999999999999996 <= edition_delta <= 0.25


Again, for $\texttt{edition}\_\texttt{delta} = 1$, many empty values of the $\texttt{edition}$ attribute can be observed. These will be marked with the special value of -1 in the data with the goal to distinguish them from the matching attribute pairs.

In [25]:
df_feature_base = dpf.mark_both_missing(df_feature_base, 'edition')

In [26]:
position = edition_delta_uniques_len

dpf.show_samples_interval(
    df_feature_base, 'edition',
    edition_delta_uniques[edition_delta_uniques_len-position-2],
    edition_delta_uniques[edition_delta_uniques_len-position-1], 10)

,duplicates,edition_delta,edition_x,edition_y
193161,0,1.0,2,2
155947,0,1.0,6,6
67368,0,1.0,2,2
100196,0,1.0,2,2
193440,0,1.0,2,2
1084,1,1.0,3,3
108748,0,1.0,4,4
1076,1,1.0,3,3
196025,0,1.0,2,2
174112,0,1.0,6,6


0.6 <= edition_delta <= 1.0


### exactDate

As discussed in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{exactDate}$ holds a year number stored in the first fout digits. Letter 'u' is used as a placeholder for an unknown digit. The attribute may hold some month and day or a second year information in its second four digits, additionally.

The attribute will be kept as a string and will not be transformed into an integer. The feature attribute of the record pair to be compared will be calculated with a modified Hamming algorithm, see appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb). The resulting similarity will be stored in a new attribute $\texttt{exactDate}\_\texttt{delta}$ which will be taken for the model calculation.

In [27]:
attribute = 'exactDate'

# Replace letter 'u' with letter 'a' for one of the two strings.
#  As an effect, the resulting Hamming similarity for a letter
#  instead of a numerical digit in either string will add with an amount 0.
df_feature_base[attribute+'_x'] = df_feature_base.exactDate_x.str.replace('u', 'a')

# Compute Hamming similarity for century string pair.
columns_metadata_dict['similarity_metrics'][attribute] = tedi.Hamming()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

# Add amount of 1/16 to Hamming similarity for every letter digit.
#  But only maximum number of letter digits in both strings of a pair.
df_feature_base[attribute+'_delta'] = df_feature_base[[
    attribute+'_x', attribute+'_y', attribute+'_delta']].apply(
    lambda x : x[attribute+'_delta'] + 
    max(x[attribute+'_x'].count('a'), x[attribute+'_y'].count('u'))/16, axis=1
)

In [28]:
df_feature_base[['exactDate_x', 'exactDate_y', 'exactDate_delta']].sample(n=10)

,exactDate_x,exactDate_y,exactDate_delta
195820,2016aaaa,2001uuuu,0.500
81887,1920aaaa,19501951,0.625
170050,1981aaaa,1963uuuu,0.500
127523,19001950,2003uuuu,0.375
42328,20091990,1991uuuu,0.250
22872,19001999,1992uuuu,0.500
28477,2004aaaa,19289999,0.250
57203,1993aaaa,2007uuuu,0.250
97512,2005aaaa,1992uuuu,0.250
147058,2004aaaa,2004uuuu,0.750


All resulting values of equal strings are equal to 1.

In [29]:
df_feature_base[['exactDate_x', 'exactDate_y', 'exactDate_delta']][
    df_feature_base.exactDate_x == df_feature_base.exactDate_y
].sort_values('exactDate_delta', ascending=False).head()

,exactDate_x,exactDate_y,exactDate_delta
159,20022000,20022000,1.0
843,19849999,19849999,1.0
845,19969999,19969999,1.0
846,19969999,19969999,1.0
847,19969999,19969999,1.0


Nine different similarity values can be found in the attribute deltas. Some sample records are shown below.

In [30]:
exactDate_deltas = np.sort(df_feature_base.exactDate_delta.unique())
exactDate_deltas

array([0.    , 0.125 , 0.25  , 0.3125, 0.375 , 0.4375, 0.5   , 0.5625,
       0.625 , 0.6875, 0.75  , 0.875 , 1.    ])

In [31]:
sample_size = 5

for i in exactDate_deltas :
    dpf.show_samples_distinct(df_feature_base, 'exactDate', i, sample_size)
    print(f'exactDate_delta = {i}')

,duplicates,exactDate_delta,exactDate_x,exactDate_y
124480,0,0.0,19819999,20172016
98281,0,0.0,19702006,20130627
77421,0,0.0,17001799,20130627
157453,0,0.0,20092005,18701880
173171,0,0.0,20159999,19282011


exactDate_delta = 0.0


,duplicates,exactDate_delta,exactDate_x,exactDate_y
215158,0,0.125,20159999,17931797
217649,0,0.125,19819999,18701880
228772,0,0.125,18901899,20151475
152777,0,0.125,19001996,20022003
208232,0,0.125,201310aa,19949999


exactDate_delta = 0.125


,duplicates,exactDate_delta,exactDate_x,exactDate_y
220203,0,0.25,2016aaaa,1979uuuu
28110,0,0.25,2004aaaa,1932uuuu
248440,0,0.25,1932aaaa,20151475
252255,0,0.25,2017aaaa,19952006
186783,0,0.25,2016aaaa,19501951


...

exactDate_delta = 0.75


,duplicates,exactDate_delta,exactDate_x,exactDate_y
227703,0,0.875,19829999,19819999
32817,0,0.875,19959999,19989999
121134,0,0.875,19849999,19949999
121216,0,0.875,19849999,19949999
121019,0,0.875,19849999,19819999


exactDate_delta = 0.875


,duplicates,exactDate_delta,exactDate_x,exactDate_y
239691,0,1.0,19911794,19911794
946,1,1.0,19001996,19001996
967,1,1.0,18901897,18901897
309,1,1.0,20071990,20071990
652,1,1.0,20092005,20092005


exactDate_delta = 1.0


### format

Due to the discussion in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{format}$ has been split up into two new attributes $\texttt{format_prefix}$ and $\texttt{format_postfix}$ which will be compared by a different similarity metrics.

- As the quality of $\texttt{format_prefix}$ is expected to be high, an identity comparison should be sufficient.
- Due to the observed structure of $\texttt{format_postfix}$, a q-gram based comparison will be chosen.

In [32]:
attribute = 'format'

columns_metadata_dict['similarity_metrics'][attribute+'_prefix'] = tedi.Identity()
columns_metadata_dict['similarity_metrics'][attribute+'_postfix'] = tedi.Jaccard(qval=2)

pfix_values = ['_prefix', '_postfix']

for pf in pfix_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+pf,
        columns_metadata_dict['similarity_metrics'][attribute+pf],
        columns_metadata_dict)

In [33]:
for i in df_feature_base.format_prefix_delta[
    df_feature_base.format_prefix_x != df_feature_base.format_prefix_y].unique():
    
    dpf.show_samples_distinct(df_feature_base, 'format_prefix', i)
    print(f'format_prefix_delta = {i}')

,duplicates,format_prefix_delta,format_prefix_x,format_prefix_y
113377,0,0.0,bk,mu
192379,0,0.0,bk,cr
114996,0,0.0,cr,bk
93897,0,0.0,mu,bk
143762,0,0.0,bk,mu


format_prefix_delta = 0.0


In [34]:
for i in df_feature_base.format_postfix_delta[
    df_feature_base.format_postfix_x != df_feature_base.format_postfix_y].unique():
    
    dpf.show_samples_distinct(df_feature_base, 'format_postfix', i)
    print(f'format_postfix_delta = {i}')

,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
208565,0,0.428571,010053,020053
104719,0,0.428571,040100,010300
179367,0,0.428571,010200,020053
154426,0,0.428571,010200,020000
117554,0,0.428571,020000,010000


format_postfix_delta = 0.4285714285714286


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
54159,0,0.111111,010300,040000
89532,0,0.111111,020000,010300
5256,0,0.111111,030600,020000
187649,0,0.111111,020053,030000
36252,0,0.111111,010100,040000


format_postfix_delta = 0.11111111111111116


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
70187,0,0.25,020000,020353
60864,0,0.25,010200,020353
108915,0,0.25,020000,020347
111298,0,0.25,020000,020353
141111,0,0.25,020000,020353


...

format_postfix_delta = 0.0


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
187966,0,1.0,020000,020000
185479,0,1.0,020000,020000
130532,0,1.0,020000,020000
122101,0,1.0,020000,020000
95158,0,1.0,010300,030100


format_postfix_delta = 1.0


,duplicates,format_postfix_delta,format_postfix_x,format_postfix_y
90454,0,0.666667,020047,020400
87943,0,0.666667,030500,030053
26102,0,0.666667,020047,020400
90470,0,0.666667,020047,020400
26086,0,0.666667,020047,020400


format_postfix_delta = 0.6666666666666666


### isbn

Swissbib uses each string element of the $\texttt{isbn}$ list separately for comparing with each string element of its comparison $\texttt{isbn}$ list. If two bibliographic units hold at least one element in common, this is interpreted as a strong indicator for duplicates [[WiCo2001](./A_References.ipynb#wico2001)].

This hard logic is used in a modified way in the context of this capstone project. A special comparison function $\texttt{.build}\_\texttt{delta}\_\texttt{isbn()}$ has been implemented that compares each list element of the left-hand side with each list element of the right-hand side of a pair. According to Swissbib's implementation, the Identity metric is used for string comparison, calculating a similarity value of 1 or 0 for each list element pair. For normalisation reasons, the sum of similarity values is divided by the number of elements of the smaller list. If both lists are empty a value of 1.0 is returned. If only one list is empty a value of 0.0 is returned.

In [35]:
attribute = 'isbn'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Identity()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

df_feature_base[attribute+'_delta'].unique()

array([1. , 0. , 0.5])

Some sample cases are shown below for each category of $\texttt{isbn_delta}$.

In [36]:
for isbn_delta_value in df_feature_base['isbn_delta'].unique():
    number_of_max_samples = min(
        10,
        len(df_feature_base[df_feature_base['isbn_delta']==isbn_delta_value])
    )

    dpf.show_samples_distinct(df_feature_base, 'isbn', isbn_delta_value, number_of_max_samples)
    print(f'isbn_delta = {isbn_delta_value}')

,duplicates,isbn_delta,isbn_x,isbn_y
191218,0,1.0,[],[]
233037,0,1.0,[],[]
135552,0,1.0,[],[]
65855,0,1.0,[],[]
228890,0,1.0,[],[]
257233,0,1.0,[],[]
250422,0,1.0,[],[]
141364,0,1.0,[],[]
73786,0,1.0,[],[]
258365,0,1.0,[],[]


isbn_delta = 1.0


,duplicates,isbn_delta,isbn_x,isbn_y
126104,0,0.0,[],[978-2-253-15933-9]
246809,0,0.0,[],[978-3-13-100294-5]
202759,0,0.0,[],"[3-15-007633-1, 3-15-027633-0]"
151121,0,0.0,[978-3-7255-6535-1],[]
13610,0,0.0,[3-932992-42-3],[]
107497,0,0.0,[3-13-127283-X],[]
17747,0,0.0,[0-521-82437-0],"[978-0-7294-1155-4, 0-7294-1155-9]"
136485,0,0.0,[0-521-82437-0],[]
69920,0,0.0,[3-8067-5097-1],[]
212129,0,0.0,[],"[978-3-598-31487-2 (print), 978-3-11-097415-7]"


isbn_delta = 0.0


,duplicates,isbn_delta,isbn_x,isbn_y
1205,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
1210,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
1202,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
161989,0,0.5,"[978-3-642-41697-2, 978-3-642-41698-9 (ebook)]","[978-3-642-41697-2, 3-642-41697-7]"
1199,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"
1195,1,0.5,"[978-3-13-127286-7, 3-13-127286-4]","[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]"
1201,1,0.5,"[978-3-13-127286-7, 978-3-13-150826-3 (PDF)]","[978-3-13-127286-7, 3-13-127286-4]"


isbn_delta = 0.5


### musicid

Chapter [Data Analysis](./1_DataAnalysis.ipynb) shows that attribute $\texttt{musicid}$ is an identifyer for a music record. A Jaccard metric has been tested on this attribute, resulting in a distribution of many high similarity values on uniques. Comparing this result with the LCS metric, the latter has been decided.

In [37]:
attribute = 'musicid'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.LCSStr()
#musicid_algorithm = tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [38]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

musicid values range [0.         0.125      0.14285714 0.16666667 0.2        0.25
 0.28571429 0.33333333 0.375      0.4        0.42857143 0.5
 0.6        0.66666667 0.75       1.        ]


In [39]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

position = uniques_len[attribute]//2

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)

,duplicates,musicid_delta,musicid_x,musicid_y
249717,0,1.0,4553,4553
66506,0,1.0,7794,7794
102258,0,1.0,4553,4553
626,1,1.0,3714,3714
780,1,1.0,502430,502430
66181,0,1.0,7794,7794
18,1,1.0,073,073
652,1,1.0,501326,501326
487,1,1.0,10425,10425
321,1,1.0,073,073


0.75 <= musicid_delta <= 1.0


,duplicates,musicid_delta,musicid_x,musicid_y
3301,0,0.0,,
119152,0,0.0,,
239725,0,0.0,,
152102,0,0.0,4553,
60852,0,0.0,,
203785,0,0.0,,
89500,0,0.0,,
39491,0,0.0,,
18815,0,0.0,4553,7918
156611,0,0.0,,


0.0 <= musicid_delta <= 0.1428571428571429


,duplicates,musicid_delta,musicid_x,musicid_y
75019,0,0.4,10425,426
245725,0,0.4,1004,10425
136178,0,0.4,601,33001
133769,0,0.4,10425,422
126584,0,0.4,10425,8421
138902,0,0.4,99036,4031
15510,0,0.4,433,92633
140292,0,0.4,10425,8421
17180,0,0.4,10425,242
234566,0,0.4,50999,99064


0.375 <= musicid_delta <= 0.4


In [40]:
dpf.show_samples_interval(
    df_feature_base[df_feature_base.duplicates==1], attribute,
    uniques[attribute][0],
    uniques[attribute][uniques_len[attribute]-1], 20)

,duplicates,musicid_delta,musicid_x,musicid_y
1226,1,0.0,,
1026,1,1.0,0630,0630
912,1,0.0,,
1086,1,0.0,,
919,1,0.0,,
1466,1,0.0,,
2,1,0.0,,
571,1,0.0,,
1195,1,0.0,,
1399,1,0.0,,


0.0 <= musicid_delta <= 1.0


The attribute is filled with a degree of below $10\%$. The chosen metric for it results in a similarity value of 1.0 for empty value pairs. This effect can be adjusted with function $\texttt{.build}\_\texttt{delta}\_\texttt{isbn()}$ as above. 

In [41]:
df_feature_base = dpf.mark_both_missing(df_feature_base, 'musicid')

### part

Three different metrics have been tried for attribute $\texttt{part}$. Finally, metric Jaro will be tested.

In [42]:
attribute = 'part'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaro()
#part_algorithm = tedi.Hamming()
#part_algorithm = tedi.LCSStr()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [43]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

part values range [0.         0.21198157 0.21483871 ... 0.95833333 0.97777778 1.        ]


In [44]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

position = uniques_len[attribute]//7

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-2],
    uniques[attribute][uniques_len[attribute]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+2], 10)

,duplicates,part_delta,part_x,part_y
76567,0,1.0,,
119030,0,1.0,,
92428,0,1.0,,
130390,0,1.0,,
77193,0,1.0,,
217131,0,1.0,,
107758,0,1.0,,
195205,0,1.0,,
146836,0,1.0,,
187665,0,1.0,,


0.9777777777777779 <= part_delta <= 1.0


,duplicates,part_delta,part_x,part_y
103935,0,0.0,,7
251655,0,0.0,"2017, nr. 313 (august 2017)",
248017,0,0.0,,"33001., 33001"
78612,0,0.0,,band 57
248158,0,0.0,,cahier d'exercices
35992,0,0.0,"werkgruppe 5, bd. 19",9
128858,0,0.0,"[42], 42",
219727,0,0.0,"63, 63",
248495,0,0.0,,"band 16, band 16"
111947,0,0.0,20008,


0.0 <= part_delta <= 0.21483870967741936


,duplicates,part_delta,part_x,part_y
142441,0,0.622222,"2620, 2620",208
227167,0,0.622222,"foglio 285, 285","bl. 23, 23,1885"
124900,0,0.622222,"2620, 2620",208
68147,0,0.622222,f. 285,"jg. 2 (1980), heft 1"
57419,0,0.622222,lfg. 2,"jg. 2 (1980), heft 1"
147644,0,0.622494,"nr. 2620, 2620","bl. 23, 23,1912, 23"
91391,0,0.622222,nr. 21,"jg. 2 (1980), heft 1"
142377,0,0.622222,"2620, 2620",208
142184,0,0.622222,"2620, 2620",208
125093,0,0.622222,"2620, 2620",208


0.6222222222222222 <= part_delta <= 0.6224937343358395


,duplicates,part_delta,part_x,part_y
228800,0,0.625000,"2, 2","2017, nr. 313, 2017, 313"
198961,0,0.625000,13 d,"1368. contemporain, 1368"
208389,0,0.623333,"28/10(2013-10), 2421-2431",2191
186368,0,0.625000,n. 1,"bl. 23,1926, 23,1926, 23"
47616,0,0.625000,"7, 7","2017, nr. 313, 2017, 313"
150315,0,0.625000,"63, 63",nr. 7633
72239,0,0.625000,bd. 57,nr. 7633
150311,0,0.625000,"63, 63",nr. 7633
199187,0,0.625000,bd. 57,nr. 7633
225185,0,0.625000,56 c,"1368. contemporain, 1368"


0.6232298474945533 <= part_delta <= 0.625


In this attribute, too, moving pairs of empty values to -1.0 will result in a clearer distinction between pairs of uniques and duplicates, as will be seen in the graphical comparison of capter [Features Discussion and Dummy Classifier Baseline](./4_FeatureDiscussionDummyBaseline.ipynb).

In [45]:
df_feature_base = dpf.mark_both_missing(df_feature_base, 'part')

### person

As a result of chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{person}$ has been split into three specific attributes. Attribute $\texttt{person}\_{100}$ and $\texttt{person}\_{700}$ hold strongly standardised string values. For comparing pure strings, a Levenshtein metric is recommended [[Chri2012](./A_References.ipynb#chri2012)]. Unfortunately, this metric shows a very long calculation time on the data of the capstone project. Comparing the similarity values of the Levenshtein metric with the similarity values of other metrics, similarity metric StrCmp95 has been decided to use.

In [46]:
attribute = 'person'

columns_metadata_dict['similarity_metrics'][attribute+'_100'] = tedi.StrCmp95()
columns_metadata_dict['similarity_metrics'][attribute+'_700'] = tedi.StrCmp95()
# tedi.Levenshtein()

pe_values = ['_100', '_700']

for pe in pe_values :
    print('Calculating person'+pe)
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+pe,
        columns_metadata_dict['similarity_metrics'][attribute+pe],
        columns_metadata_dict)

Calculating person_100
Calculating person_700


In [47]:
pe = '_100'

uniques[attribute+pe], uniques_len[attribute+pe] = dpf.determine_similarity_values(
    df_feature_base, attribute+pe)

person_100 values range [0.         0.29487179 0.30092593 ... 0.95384615 0.96       1.        ]


In [48]:
position = uniques_len[attribute+pe]

dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position-2],
    uniques[attribute+pe][uniques_len[attribute+pe]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position],
    uniques[attribute+pe][uniques_len[attribute+pe]-position+2], 10)

,duplicates,person_100_delta,person_100_x,person_100_y
243457,0,1.0,,
49353,0,1.0,,
244008,0,1.0,,
113394,0,1.0,,
196522,0,1.0,,
121468,0,1.0,,
6054,0,1.0,,
123603,0,1.0,,
211826,0,1.0,,
165136,0,1.0,,


0.96 <= person_100_delta <= 1.0


,duplicates,person_100_delta,person_100_x,person_100_y
179613,0,0.0,,mozartwolfgang amadeus
232910,0,0.0,,moriartyliane
171286,0,0.0,voltaire1694-1778,
166634,0,0.0,,mozartwolfgang amadeus1756-1791(de-588)1185845...
171347,0,0.0,voltaire1694-1778,
194331,0,0.0,broderursula1987-(de-588)1095569961verfasseraut,
111863,0,0.0,austenjane1775-1817(de-588)118505173,
39196,0,0.0,,mozartwolfgang amadeus1756-1791(de-588)118584596
170136,0,0.0,mozartwolfgang amadeus,
64121,0,0.0,,austenjane


0.0 <= person_100_delta <= 0.3009259259259258


For comparing person names, like in attribute $\texttt{person}\_{245c}$, a Jaro metric will be tested [[Chri2012](./A_References.ipynb#chri2012)].

In [49]:
pe = '_245c'

columns_metadata_dict['similarity_metrics'][attribute+pe] = tedi.Jaro()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute+pe,
    columns_metadata_dict['similarity_metrics'][attribute+pe],
    columns_metadata_dict)

In [50]:
uniques[attribute+pe], uniques_len[attribute+pe] = dpf.determine_similarity_values(
    df_feature_base, attribute+pe)

person_245c values range [0.         0.24632035 0.25462963 ... 0.99603175 0.99863388 1.        ]


In [51]:
position = uniques_len[attribute+pe]

dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position-2],
    uniques[attribute+pe][uniques_len[attribute+pe]-position-1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+pe,
    uniques[attribute+pe][uniques_len[attribute+pe]-position],
    uniques[attribute+pe][uniques_len[attribute+pe]-position+2], 10)

,duplicates,person_245c_delta,person_245c_x,person_245c_y
4641,0,1.0,,
231511,0,1.0,,
32852,0,1.0,,
226743,0,1.0,,
68235,0,1.0,,
103,1,1.0,andreas flury,andreas flury
211652,0,1.0,,
274,1,1.0,ein film von volker schlöndorff ; nach dem rom...,ein film von volker schlöndorff ; nach dem rom...
204301,0,1.0,,
4590,0,1.0,,


0.9986338797814208 <= person_245c_delta <= 1.0


,duplicates,person_245c_delta,person_245c_x,person_245c_y
209985,0,0.0,,w.a. mozart ; libretto: emanuel schikaneder ; ...
72858,0,0.0,"w. a. mozart ; klavierauszug, nach dem in der ...",
206978,0,0.0,"[virginie rozot, amelio patrizia, selena vigan...",
32764,0,0.0,,"mortzfeld, peter; raabe, paul"
257050,0,0.0,g.h. dufour direxit ; h. müllhaupt sculpsit,
87878,0,0.0,,sigrid kessler... [et al.] ; [hrsg.:] interkan...
152627,0,0.0,w. a. mozart ; texte de schikaneder,
88200,0,0.0,,"mortzfeld, peter; raabe, paul"
182101,0,0.0,,[sigrid kessler... et al.] ; [hrsg. interkanto...
201615,0,0.0,,von w.a. mozart ; dichtung nach ludwig gieseck...


0.0 <= person_245c_delta <= 0.25462962962962954


The similarities of all three $\texttt{person}$ attributes are affected by empty values. These will be handled as for the attributes above.

In [52]:
pe_values = ['_100', '_245c', '_700']

for pe in pe_values :
    df_feature_base = dpf.mark_both_missing(df_feature_base, 'person'+pe)

### pubinit

This attribute holds publisher strings that have a similar representation as attribute $\texttt{person}$ above. A Jaro metric will be used.

In [53]:
attribute = 'pubinit'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaro()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [54]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

pubinit values range [0.         0.25132275 0.25303644 ... 0.98404165 0.98484848 1.        ]


In [55]:
position = uniques_len[attribute]//3

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position-1],
    uniques[attribute][uniques_len[attribute]-position], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-5],
    uniques[attribute][uniques_len[attribute]-1], 10)

,duplicates,pubinit_delta,pubinit_x,pubinit_y
190322,0,0.554834,frenetic films,staatlicher lehrmittelverlag bern
106286,0,0.554834,frenetic films,staatlicher lehrmittelverlag bern
222322,0,0.554809,cambridge univ. press,brillant classics
99134,0,0.554834,frenetic films,staatlicher lehrmittelverlag bern
87035,0,0.554834,[phonogram],polydor international
139066,0,0.554834,frenetic films,staatlicher lehrmittelverlag bern
164098,0,0.554834,frenetic films,staatlicher lehrmittelverlag bern
142046,0,0.554834,philipp reclam,staatlicher lehrmittelverlag bern
97168,0,0.554834,gerstenberg,pearson education ltd
256478,0,0.554834,frenetic films,staatlicher lehrmittelverlag bern


0.5548085901027077 <= pubinit_delta <= 0.5548340548340548


,duplicates,pubinit_delta,pubinit_x,pubinit_y
16787,0,1.0,,
69158,0,1.0,,
59720,0,1.0,,
208136,0,1.0,,
189982,0,1.0,,
28040,0,1.0,,
241505,0,1.0,,
21579,0,1.0,,
780,1,1.0,kinowelt home entertainment,kinowelt home entertainment
24805,0,1.0,,


0.9777777777777779 <= pubinit_delta <= 1.0


The similarities of all three $\texttt{pubinit}$ attributes are affected by empty values. These will be handled as for the attributes above.

In [56]:
df_feature_base = dpf.mark_both_missing(df_feature_base, attribute)

### scale

Comparing the similarity metrics of some sample value pairs of attribute $\texttt{scale}$ in appendix [Comparison of Similarity Metrics](./B_CompareSimilarities.ipynb), a Jaccard metrics has been identifyed to express the best matching behaviour for purely numerical values stored in the attribute.

In [57]:
attribute = 'scale'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaccard()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [58]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

scale values range [0.         0.04587156 0.05504587 0.57142857 1.        ]


In [59]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-3],
    uniques[attribute][uniques_len[attribute]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-4],
    uniques[attribute][uniques_len[attribute]-3], 10)

,duplicates,scale_delta,scale_x,scale_y
67165,0,0.0,,100000
151801,0,0.0,,100000
64206,0,0.0,,100000
122610,0,0.0,,100000
172934,0,0.0,,100000
257006,0,0.0,100000,
188150,0,0.0,,100000
2205,0,0.0,,100000
22478,0,0.0,,100000
95052,0,0.0,50000,


0.0 <= scale_delta <= 0.04587155963302747


,duplicates,scale_delta,scale_x,scale_y
68035,0,0.571429,50000,100000
53731,0,0.571429,50000,100000
227168,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
182214,0,0.571429,50000,100000
68370,0,0.571429,50000,100000
227497,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
95183,0,0.571429,50000,100000
94854,0,0.571429,50000,100000
182196,0,0.571429,50000,100000
227490,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000


0.05504587155963303 <= scale_delta <= 0.5714285714285714


,duplicates,scale_delta,scale_x,scale_y
227174,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
227489,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
227499,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
871,1,0.045872,50000,Scala 1:50.000 ; proiezione cilindrica ad asse...
227496,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
874,1,0.045872,Scala 1:50.000 ; proiezione cilindrica ad asse...,50000
227171,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
872,1,0.045872,Scala 1:50.000 ; proiezione cilindrica ad asse...,50000
227168,0,0.055046,Scala 1:50.000 ; proiezione cilindrica ad asse...,100000
873,1,0.045872,Scala 1:50.000 ; proiezione cilindrica ad asse...,50000


0.04587155963302747 <= scale_delta <= 0.05504587155963303


Attribute $\texttt{scale}$ is filled for maps, only. Due to its sparse filling, the similarities of the attribute is affected strongly by empty values. These empty values will be marked with the special value of -1.0.

In [60]:
df_feature_base = dpf.mark_both_missing(df_feature_base, attribute)

### ttlfull

Due to the discussion in chapter [Data Analysis](./1_DataAnalysis.ipynb), attribute $\texttt{ttlfull}$ has been split up into two new attributes $\texttt{ttlfull_245}$ and $\texttt{ttlfull_246}$ which will be compared by the same similarity metrics. A visual analysis of the values stored in the attribute, reveals a string of words, comparable to the strings in attribute $\texttt{person_245c}$, above. The same similarity metric will be used for both title attributes, therefore.

In [61]:
attribute = 'ttlfull'

columns_metadata_dict['similarity_metrics'][attribute+'_245'] = tedi.Jaro()
columns_metadata_dict['similarity_metrics'][attribute+'_246'] = tedi.Jaro()

tf_values = ['_245', '_246']

for tf in tf_values :
    df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
        df_feature_base, attribute+tf,
        columns_metadata_dict['similarity_metrics'][attribute+tf],
        columns_metadata_dict)

In [62]:
for tf in tf_values :
    uniques[attribute+tf], uniques_len[attribute+tf] = dpf.determine_similarity_values(
        df_feature_base, attribute+tf)

ttlfull_245 values range [0.         0.24175824 0.24358974 ... 0.99555556 0.99747475 1.        ]
ttlfull_246 values range [0.         0.27407407 0.35555556 0.36111111 0.36366612 0.36666667
 0.39444444 0.39814815 0.40238095 0.40555556 0.41140642 0.41190476
 0.41296296 0.41313131 0.4168847  0.41923077 0.41931217 0.42380952
 0.42424242 0.42484886 0.42767857 0.43003072 0.43333333 0.43758865
 0.43913043 0.43998145 0.44198966 0.44242424 0.44507736 0.44539295
 0.44722222 0.4479837  0.44848485 0.44901497 0.45       0.45011338
 0.45277778 0.45436508 0.45610717 0.45714286 0.45764895 0.45833333
 0.46094276 0.46205962 0.46527102 0.46729958 0.46851852 0.46885522
 0.46944444 0.46945938 0.47130647 0.47142857 0.47162447 0.47222222
 0.472318   0.47267613 0.47285513 0.47308219 0.47356964 0.47372609
 0.47388167 0.47417027 0.47428613 0.47431581 0.47446677 0.47462215
 0.47491364 0.47508251 0.47515152 0.47518634 0.47597067 0.47606838
 0.47613256 0.47629884 0.47642654 0.47794389 0.47829216 0.47836071
 0.4788

In [63]:
tf = '_245'
position = uniques_len[attribute+tf]

dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-position],
    uniques[attribute+tf][uniques_len[attribute+tf]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-3],
    uniques[attribute+tf][uniques_len[attribute+tf]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute+tf,
    uniques[attribute+tf][uniques_len[attribute+tf]-4],
    uniques[attribute+tf][uniques_len[attribute+tf]-3], 10)

,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
77308,0,0.0,"genéve, genff",domodossola
111810,0,0.0,emma,blick in die welt
218882,0,0.0,bildungsforschung und bildungspraxis,emma
36382,0,0.0,bildungsforschung und bildungspraxis,emma
32537,0,0.0,arts,blick in die welt
32553,0,0.0,arts,blick in die welt
11694,0,0.0,emma,blick in die welt
185717,0,0.0,emma,blick in die welt
107041,0,0.0,emma,blick in die welt
2766,0,0.0,emma,blick in die welt


0.0 <= ttlfull_245_delta <= 0.2417582417582418


,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
249,1,0.997475,sozialleistungsbetrug: sozialversicherungsbetr...,sozialleistungsbetrug: sozialversicherungsbetr...
237,1,0.997475,sozialleistungsbetrug: sozialversicherungsbetr...,sozialleistungsbetrug: sozialversicherungsbetr...
143980,0,0.995556,"bonne chance !, cours de langue française deux...","bonne chance !, cours de langue française, deu..."


0.9955555555555556 <= ttlfull_245_delta <= 0.9974747474747474


,duplicates,ttlfull_245_delta,ttlfull_245_x,ttlfull_245_y
93,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
193301,0,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
100,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
51453,0,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
1360,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
51792,0,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
128080,0,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
88,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
91,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."
66,1,0.99537,"der moralische status der tiere, henry salt, p...","der moralische status der tiere, henry salt, p..."


0.9953703703703703 <= ttlfull_245_delta <= 0.9955555555555556


Attribute $\texttt{ttlfull}\_\texttt{245}$ is filled for all data rows of Swissbib's raw data as can be seen in chapter [Data Analysis](./1_DataAnalysis.ipynb). For attribute $\texttt{ttlfull}\_\texttt{245}$, the filling is below $10\%$. The data pairs with missing values on both sides will be marked with a value of -1.0 as above.

In [64]:
df_feature_base = dpf.mark_both_missing(df_feature_base, attribute+'_246')

### volumes

This attribute is described in chapter [Data Analysis](./1_DataAnalysis.ipynb) to hold a kind of contents that resembles the contents of attribute $\texttt{part}$. The same similarity metrics will be used for attribute $\texttt{volumes}$ as for attriubte $\texttt{part}$, therefore.

In [65]:
attribute = 'volumes'

columns_metadata_dict['similarity_metrics'][attribute] = tedi.Jaro()
# tedi.MongeElkan()

df_feature_base, columns_metadata_dict = dpf.build_delta_feature(
    df_feature_base, attribute,
    columns_metadata_dict['similarity_metrics'][attribute],
    columns_metadata_dict)

In [66]:
uniques[attribute], uniques_len[attribute] = dpf.determine_similarity_values(
    df_feature_base, attribute)

volumes values range [0.         0.24175824 0.24691358 ... 0.98484848 0.98765432 1.        ]


In [67]:
position = uniques_len[attribute]

dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-position],
    uniques[attribute][uniques_len[attribute]-position+1], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-3],
    uniques[attribute][uniques_len[attribute]-2], 10)
dpf.show_samples_interval(
    df_feature_base, attribute,
    uniques[attribute][uniques_len[attribute]-4],
    uniques[attribute][uniques_len[attribute]-3], 10)

,duplicates,volumes_delta,volumes_x,volumes_y
47441,0,0.0,1 partitur ([448] s.),
145159,0,0.0,134 p.,
75010,0,0.0,"1 klavierauszug (vi, 188 s.)",
74028,0,0.0,1 klavierauszug (221 s.),
167661,0,0.0,,1 band
256350,0,0.0,1 dvd-video (82 min.),
208578,0,0.0,,"1 online resource (iv, 361 p.)"
88378,0,0.0,,1 klavierauszug (188 s.)
59749,0,0.0,1 textbuch,
111818,0,0.0,600 s.,


0.0 <= volumes_delta <= 0.2417582417582418


,duplicates,volumes_delta,volumes_x,volumes_y
304,1,0.987654,2 dvd-video (ca. 109 min.),2 dvd-videos (ca. 109 min.)
100860,0,0.984848,1 dvd-video (169 min.),1 dvd-video (16 min.)
73444,0,0.984848,1 dvd-video (169 min.),1 dvd-video (16 min.)
286,1,0.987654,2 dvd-video (ca. 109 min.),2 dvd-videos (ca. 109 min.)
274,1,0.987654,2 dvd-videos (ca. 109 min.),2 dvd-video (ca. 109 min.)
295,1,0.987654,2 dvd-videos (ca. 109 min.),2 dvd-video (ca. 109 min.)
271,1,0.987654,2 dvd-videos (ca. 109 min.),2 dvd-video (ca. 109 min.)
283,1,0.987654,2 dvd-video (ca. 109 min.),2 dvd-videos (ca. 109 min.)
292,1,0.987654,2 dvd-videos (ca. 109 min.),2 dvd-video (ca. 109 min.)
307,1,0.987654,2 dvd-video (ca. 109 min.),2 dvd-videos (ca. 109 min.)


0.9848484848484849 <= volumes_delta <= 0.9876543209876543


,duplicates,volumes_delta,volumes_x,volumes_y
100860,0,0.984848,1 dvd-video (169 min.),1 dvd-video (16 min.)
148669,0,0.978495,2 disques compacts en 1 coffret,3 disques compacts en 1 coffret
73444,0,0.984848,1 dvd-video (169 min.),1 dvd-video (16 min.)


0.978494623655914 <= volumes_delta <= 0.9848484848484849


Attribute $\texttt{volumes}$ holds rows with missing data. The data pairs with missing values on both sides will be marked with a value of -1.0.

In [68]:
df_feature_base = dpf.mark_both_missing(df_feature_base, attribute)

## Feature Base

The metris for each attribute of the feature DataFrame has been decided and the features have been calculated. The columns with the original attribute values are not needed for further processing and they will be dropped to generate the feature matrix for modelling the estimators.

In [69]:
# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(df_feature_base.columns)

df_feature_base.head(20)

,duplicates,coordinate_E_x,coordinate_E_y,coordinate_N_x,coordinate_N_y,corporate_full_x,corporate_full_y,doi_x,doi_y,edition_x,edition_y,exactDate_x,exactDate_y,format_prefix_x,format_prefix_y,format_postfix_x,format_postfix_y,isbn_x,isbn_y,musicid_x,musicid_y,part_x,part_y,person_100_x,person_100_y,person_700_x,person_700_y,person_245c_x,person_245c_y,pubinit_x,pubinit_y,scale_x,scale_y,ttlfull_245_x,ttlfull_245_y,ttlfull_246_x,ttlfull_246_y,volumes_x,volumes_y,coordinate_E_delta,coordinate_N_delta,corporate_full_delta,doi_delta,edition_delta,exactDate_delta,format_prefix_delta,format_postfix_delta,isbn_delta,musicid_delta,part_delta,person_100_delta,person_700_delta,person_245c_delta,pubinit_delta,scale_delta,ttlfull_245_delta,ttlfull_246_delta,volumes_delta
0,1,,,,,,,,,,,2009aaaa,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane1775-1817(de-588)118505173,"grawechristian, graweursula","grawechristian, graweursula",jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem englischen übersetzt von...,reclam jun.,reclam jun.,,,"emma, roman","emma, roman",,,600 s.,600 s.,-1.0,-1.0,-1.000000,-1.0,-1.0,0.75,1.0,1.0,1.0,-1.0,1.0,1.000000,1.000000,1.000000,1.000000,-1.0,1.000000,-1.0,1.000000
1,1,,,,,,,,,,,2009aaaa,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane1775-1817(de-588)118505173,"grawechristian, graweursula",,jane austen ; aus dem englischen übersetzt von...,jane austen ; aus dem engl. übers. von ursula ...,reclam jun.,reclam,,,"emma, roman",emma,,,600 s.,600 s.,-1.0,-1.0,-1.000000,-1.0,-1.0,0.75,1.0,1.0,1.0,-1.0,1.0,1.000000,0.000000,0.818905,0.848485,-1.0,0.787879,-1.0,1.000000
2,1,,,,,,,,,,,2009aaaa,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane,"grawechristian, graweursula",,jane austen ; aus dem englischen übersetzt von...,jane austen,reclam jun.,reclam,,,"emma, roman","emma, roman",,,600 s.,600 s.,-1.0,-1.0,-1.000000,-1.0,-1.0,0.75,1.0,1.0,1.0,-1.0,1.0,0.855556,0.000000,0.697740,0.848485,-1.0,1.000000,-1.0,1.000000
3,1,,,,,,,,,,,2009aaaa,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane1775-1817(de-588)118505173,,"grawechristian, graweursula",jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem englischen übersetzt von...,reclam,reclam jun.,,,emma,"emma, roman",,,600 s.,600 s.,-1.0,-1.0,-1.000000,-1.0,-1.0,0.75,1.0,1.0,1.0,-1.0,1.0,1.000000,0.000000,0.818905,0.848485,-1.0,0.787879,-1.0,1.000000
4,1,,,,,,,,,,,2009aaaa,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane1775-1817(de-588)118505173,,,jane austen ; aus dem engl. übers. von ursula ...,jane austen ; aus dem engl. übers. von ursula ...,reclam,reclam,,,emma,emma,,,600 s.,600 s.,-1.0,-1.0,-1.000000,-1.0,-1.0,0.75,1.0,1.0,1.0,-1.0,1.0,1.000000,-1.000000,1.000000,1.000000,-1.0,1.000000,-1.0,1.000000
5,1,,,,,,,,,,,2009aaaa,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],,,20008,20008,austenjane1775-1817(de-588)118505173,austenjane,,,jane austen ; aus dem engl. übers. von ursula ...,jane austen,reclam,reclam,,,emma,"emma, roman",,,600 s.,600 s.,-1.0,-1.0,-1.000000,-1.0,-1.0,0.75,1.0,1.0,1.0,-1.0,1.0,0.855556,-1.000000,0.702265,1.000000,-1.0,0.787879,-1.0,1.000000
6,1,,,,,,,,,,,2009aaaa,2009uuuu,bk,bk,020000,020000,[978-3-15-020008-7],[978-3-15-020008-7],,,20008,20008,austenjane,austenjane1775-1817(de-588)118505173,,"grawechristian, graweursula",jane austen,jane austen ; aus dem englischen übersetzt von...,reclam,reclam jun.,,,"emma, roman","emma, roman",,,600 s.,600 s.,-1.0,-1.0,-1.000000,-1.0,-1.0,0.75,1.0,1.0,1.0,-1.0,1.0,0.855556,0.000000,0.697740,0.848485,-1.0,1.000000,-1.0,1.000000
7,1,,,,,,,,,,,2009aaaa,20

In [70]:
# Drop all non-delta columns, except of 'duplicates'
columns_to_be_dropped = [e for e in columns_metadata_dict['columns_to_use']
                         if e != 'duplicates']

df_feature_base.drop(columns=columns_to_be_dropped, inplace=True)

In [71]:
for i in range(2):
    display(df_feature_base[df_feature_base.duplicates==i].sample(n=20))

,duplicates,coordinate_E_delta,coordinate_N_delta,corporate_full_delta,doi_delta,edition_delta,exactDate_delta,format_prefix_delta,format_postfix_delta,isbn_delta,musicid_delta,part_delta,person_100_delta,person_700_delta,person_245c_delta,pubinit_delta,scale_delta,ttlfull_245_delta,ttlfull_246_delta,volumes_delta
62233,0,-1.0,-1.0,-1.0,-1.0,-1.0,0.500,1.0,0.428571,1.0,0.0,0.000000,0.000000,0.000000,0.620253,0.000000,-1.0,0.576173,0.0,0.386772
102516,0,-1.0,-1.0,-1.0,-1.0,-1.0,0.250,0.0,0.111111,0.0,0.0,-1.000000,0.000000,0.000000,0.439634,0.477041,-1.0,0.566667,-1.0,0.470085
222441,0,-1.0,-1.0,-1.0,-1.0,-1.0,0.250,0.0,0.111111,0.0,-1.0,-1.000000,0.000000,0.591453,0.541867,0.000000,-1.0,0.512346,-1.0,0.492063
200672,0,0.0,0.0,0.0,-1.0,-1.0,0.250,0.0,0.111111,0.0,-1.0,0.646825,0.000000,0.509524,0.522370,0.542857,0.0,0.532348,0.0,0.447619
224093,0,-1.0,-1.0,-1.0,-1.0,0.0,0.625,1.0,0.428571,0.0,-1.0,0.000000,0.486108,0.000000,0.463810,0.000000,-1.0,0.503141,-1.0,0.377778
40057,0,-1.0,-1.0,-1.0,-1.0,-1.0,0.375,0.0,0.111111,1.0,-1.0,0.000000,1.000000,-1.000000,0.589576,0.000000,-1.0,0.754852,-1.0,0.000000
153788,0,-1.0,-1.0,-1.0,0.0,0.0,0.500,1.0,0.111111,0.0,-1.0,-1.000000,0.000000,0.550772,0.534510,0.000000,-1.0,0.512945,-1.0,0.000000
85360,0,0.0,0.0,0.0,-1.0,-1.0,0.375,0.0,0.428571,0.0,-1.0,0.000000,-1.000000,0.000000,0.000000,-1.000000,0.0,0.438889,-1.0,0.000000
13316,0,-1.0,-1.0,-1.0,-1.0,-1.0,0.625,0.0,0.428571,0.0,0.0,0.000000,0.000000,0.000000,0.349383,0.602239,-1.0,0.577778,-1.0,0.479798
119339,0,-1.0,-1.0,-1.0,-1.0,-1.0,0.500,0.0,0.111111,1.0,0.0,0.000000,0.000000,0.000000,0.630007,0.000000,-1.0,0.591116,-1.0,0.532258


,duplicates,coordinate_E_delta,coordinate_N_delta,corporate_full_delta,doi_delta,edition_delta,exactDate_delta,format_prefix_delta,format_postfix_delta,isbn_delta,musicid_delta,part_delta,person_100_delta,person_700_delta,person_245c_delta,pubinit_delta,scale_delta,ttlfull_245_delta,ttlfull_246_delta,volumes_delta
71,1,-1.0,-1.0,-1.000000,-1.0,-1.0,0.75,1.0,1.000000,1.0,-1.0,1.000000,0.870588,-1.000000,1.000000,-1.000000,-1.000000,1.000000,-1.0,1.000000
1421,1,-1.0,-1.0,1.000000,-1.0,-1.0,0.75,1.0,1.000000,1.0,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,1.000000,1.0,1.000000
1158,1,-1.0,-1.0,0.836822,-1.0,-1.0,0.50,1.0,1.000000,1.0,0.2,-1.000000,1.000000,0.565038,0.432367,0.757576,-1.000000,0.805556,-1.0,0.548693
631,1,-1.0,-1.0,-1.000000,-1.0,-1.0,0.50,1.0,1.000000,1.0,0.0,-1.000000,1.000000,0.000000,0.000000,-1.000000,-1.000000,0.794613,0.0,0.820106
944,1,-1.0,-1.0,-1.000000,-1.0,-1.0,0.50,1.0,0.428571,1.0,0.0,-1.000000,0.891667,0.000000,0.000000,0.000000,-1.000000,0.766453,0.0,0.629461
183,1,-1.0,-1.0,1.000000,-1.0,-1.0,1.00,1.0,1.000000,1.0,-1.0,-1.000000,-1.000000,-1.000000,0.673360,0.000000,-1.000000,0.798535,0.0,-1.000000
302,1,-1.0,-1.0,-1.000000,-1.0,-1.0,0.75,1.0,1.000000,1.0,1.0,-1.000000,-1.000000,0.624096,0.830688,0.000000,-1.000000,1.000000,-1.0,0.779915
1455,1,-1.0,-1.0,-1.000000,-1.0,-1.0,0.75,1.0,1.000000,1.0,-1.0,0.713580,-1.000000,0.000000,0.000000,-1.000000,-1.000000,1.000000,0.0,1.000000
206,1,-1.0,-1.0,1.000000,-1.0,-1.0,1.00,1.0,1.000000,1.0,-1.0,-1.000000,-1.000000,-1.000000,0.673360,-1.000000,-1.000000,0.798535,0.0,0.000000
753,1,-1.0,-1.0,1.000000,-1.0,-1.0,1.00,1.0,1.000000,1.0,-1.0,-1.000000,-1.000000,-1.000000,1.000000,1.000000,-1.000000,1.000000,1.0,1.000000


## Summary

The following table lists the metric used for each attribute. Attributes with the same font color indicate similar types of filling.

In [72]:
columns_metadata_dict['similarity_metrics']

{'coordinate_E': LCSStr({'qval': 1, 'external': True}),
 'coordinate_N': LCSStr({'qval': 1, 'external': True}),
 'corporate_full': StrCmp95({'long_strings': False, 'external': True}),
 'doi': Identity({'qval': 1, 'external': True}),
 'edition': Jaccard({'qval': 1, 'as_set': False, 'external': True}),
 'exactDate': Hamming({'qval': 1, 'test_func': <function Base._ident at 0x1232d8f28>, 'truncate': False, 'external': True}),
 'format_prefix': Identity({'qval': 1, 'external': True}),
 'format_postfix': Jaccard({'qval': 2, 'as_set': False, 'external': True}),
 'isbn': Identity({'qval': 1, 'external': True}),
 'musicid': LCSStr({'qval': 1, 'external': True}),
 'part': Jaro({'qval': 1, 'long_tolerance': False, 'winklerize': False, 'external': True}),
 'person_100': StrCmp95({'long_strings': False, 'external': True}),
 'person_700': StrCmp95({'long_strings': False, 'external': True}),
 'person_245c': Jaro({'qval': 1, 'long_tolerance': False, 'winklerize': False, 'external': True}),
 'pubinit'

| attribute     | subtype | description | similarity metric |
| ------------- |:--------|:------------|:------------------|
|<font color='red'>[coordinate](#coordinate)</font>|<font color='red'>\_E</font>|<font color='red'>Code</font>|<font color='red'>LCSStr</font>|
|               |<font color='red'>\_N</font>|<font color='red'>Code</font>|<font color='red'>LCSStr</font>|
|<font color='blue'>[corporate](#corporate)</font>|<font color='blue'>\_full</font>|<font color='blue'>Names</font>|<font color='blue'>StrCmp95</font>|
|<font color='green'>[doi](#doi)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='orange'>[edition](#edition)</font>|         |<font color='orange'>Numbers</font>|<font color='orange'>Jaccard</font>|
|<font color='brown'>[exactDate](#exactDate)</font>|         |<font color='brown'>Date</font>|<font color='brown'>Hamming</font>|
|<font color='black'>[format](#format)</font>|<font color='black'>\_prefix</font>|<font color='black'>Code(2)</font>|<font color='black'>Identity</font>|
|               |<font color='black'>\_postfix</font>|<font color='black'>Code(6)</font>|<font color='black'>Jaccard (qval=2)</font>|
|<font color='green'>[isbn](#isbn)</font>|         |<font color='green'>Identifier</font>|<font color='green'>Identity</font>|
|<font color='green'>[musicid](#musicid)</font>|         |<font color='green'>Identifier</font>|<font color='green'>LCSStr</font>|
|<font color='orange'>[part](#part)</font>|         |<font color='orange'>Numbers</font>|<font color='orange'>Jaro</font>|
|<font color='blue'>[person](#person)</font>|<font color='blue'>\_100</font>|<font color='blue'>Names</font>|<font color='blue'>StrCmp95</font>|
|               |<font color='blue'>\_700</font>|<font color='blue'>Names</font>|<font color='blue'>StrCmp95</font>|
|               |<font color='blue'>\_245c</font>|<font color='blue'>Names</font>|<font color='blue'>Jaro</font>|
|<font color='blue'>[pubinit](#pubinit)</font>|         |<font color='blue'>Names</font>|<font color='blue'>Jaro</font>|
|<font color='green'>[scale](#scale)</font>|         |<font color='green'>Code</font>|<font color='green'>Jaccard</font>|
|<font color='blue'>[ttlfull](#ttlfull)</font>|<font color='blue'>\_245</font>|<font color='blue'>String</font>|<font color='blue'>Jaro</font>|
|               |<font color='blue'>\_246</font>|<font color='blue'>String</font>|<font color='blue'>Jaro</font>|
|<font color='orange'>[volumes](#volumes)</font>|         |<font color='orange'>Numbers</font>|<font color='orange'>Jaro</font>|

## Feature Matrix and Target Vector Handover

To hand over the resulting DataFrame of this chapter, the DataFrame is saved into a pickle file that will be read in the next chapters [Decision Tree Model](./5_DecisionTreeModel.ipynb), ... as input file.

In [73]:
import pickle as pk

# Binary intermediary file
with open(os.path.join(path_goldstandard,
                       'labelled_feature_matrix.pkl'), 'wb') as df_output_file:
    pk.dump(df_feature_base, df_output_file)